In [14]:
import pandas as pd
import time
import datetime

In [15]:
        import pickle
        with open('./csfail_history.pkl', 'rb') as f:
            result = pickle.load(f)
        print(result)

defaultdict(<class 'list'>, {'3910287': ['x1.00', '46', '113.79', '23:30', '15-05-2023', ['Authorization', 'ROUND', '#3910287', 'HASH', 'e7cc834cb881aff12f20c8546e2dcf8cb1c309a501515f674b9e2ba5fd7b2302', 'SEED:', 'kGNzEHsEBqwO0IZshGSv2z1Zam19pxB4K7RZBKzKrggN', 'CHECK', '46', 'PLAYERS', '113.79', 'BANK', '61', 'SKINS', '13', '+1'], ['X1.2', '10.55', '+1'], ['X2.49', '7.34'], ['X1.19', '7.28'], ['X1.2', '6', '+4'], ['X1.5', '5.18'], ['Authorization', 'ROUND', '#3910287', 'HASH', 'e7cc834cb881aff12f20c8546e2dcf8cb1c309a501515f674b9e2ba5fd7b2302', 'SEED:', 'kGNzEHsEBqwO0IZshGSv2z1Zam19pxB4K7RZBKzKrggN', 'CHECK', '46', 'PLAYERS', '113.79', 'BANK', '61', 'SKINS', '13', '+1'], ['X1.2', '10.55', '+1'], ['X2.49', '7.34'], ['X1.19', '7.28'], ['X1.2', '6', '+4'], ['X1.5', '5.18']], '3910286': ['x5.02', '53', '170.7', '23:29', '15-05-2023', ['Authorization', 'ROUND', '#3910286', 'HASH', '4547a0600997792caed94366394fb000d76d44bb0c0e2619201b77bb90206d9a', 'SEED:', 'i0UYCdbWttCeTgOQ7TSDoh5G8aGNn701SY

In [16]:
for i in result['3911148']:
    print(type(i))
    print(i)
    print("=========================================================")

<class 'str'>
x21.18
<class 'str'>
86
<class 'str'>
293.25
<class 'str'>
05:13
<class 'str'>
15-05-2023
<class 'list'>
['Authorization', 'ROUND', '#3911148', 'HASH', 'ad4a827b57eee3e931bf49397ebdf4cbbd1eaefb53de906da55a1ca79163c6b4', 'SEED:', 'miv7IYVlZ7suP6e5pdRZH0FZcvJcolRMiPI4kxNoroke', 'CHECK', '86', 'PLAYERS', '293.25', 'BANK', '117', 'SKINS', '29.1', '34.92']
<class 'list'>
['X1.2', '15.88', '+1', '18.74']
<class 'list'>
['X1.18', '10.69', '13.77']
<class 'list'>
['X1.29', '10.35', '+1', '36.08']
<class 'list'>
['X3.5', '8.93', '10.72']
<class 'list'>
['X1.2', '8.9', '18.96']
<class 'list'>
['Authorization', 'ROUND', '#3911148', 'HASH', 'ad4a827b57eee3e931bf49397ebdf4cbbd1eaefb53de906da55a1ca79163c6b4', 'SEED:', 'miv7IYVlZ7suP6e5pdRZH0FZcvJcolRMiPI4kxNoroke', 'CHECK', '86', 'PLAYERS', '293.25', 'BANK', '117', 'SKINS', '29.1', '34.92']
<class 'list'>
['X1.2', '15.88', '+1', '18.74']
<class 'list'>
['X1.18', '10.69', '13.77']
<class 'list'>
['X1.29', '10.35', '+1', '36.08']
<class 

In [18]:
l = [datetime.datetime(result[i][3]) for i in result.keys()]
#list(map(lambda x: x.values()[3], l))

TypeError: an integer is required (got type str)

In [12]:
l

['23:30',
 '23:29',
 '23:29',
 '23:28',
 '23:28',
 '23:28',
 '23:27',
 '23:27',
 '23:27',
 '23:27',
 '23:26',
 '23:26',
 '23:25',
 '23:25',
 '23:24',
 '23:24',
 '23:24',
 '23:23',
 '23:23',
 '23:22',
 '23:22',
 '23:22',
 '23:21',
 '23:21',
 '23:21',
 '23:20',
 '23:20',
 '23:20',
 '23:19',
 '23:19',
 '23:18',
 '23:18',
 '23:17',
 '23:17',
 '23:17',
 '23:16',
 '23:16',
 '23:15',
 '23:15',
 '23:15',
 '23:14',
 '23:14',
 '23:13',
 '23:13',
 '23:13',
 '23:12',
 '23:11',
 '23:11',
 '23:10',
 '23:10',
 '23:09',
 '23:09',
 '23:09',
 '23:09',
 '23:08',
 '23:08',
 '23:08',
 '23:07',
 '23:07',
 '23:07',
 '23:06',
 '23:06',
 '23:06',
 '23:05',
 '23:05',
 '23:05',
 '23:05',
 '23:05',
 '23:04',
 '23:04',
 '23:04',
 '23:03',
 '23:03',
 '23:03',
 '23:02',
 '23:02',
 '23:02',
 '23:01',
 '23:01',
 '23:01',
 '23:00',
 '23:00',
 '22:59',
 '22:59',
 '22:58',
 '22:58',
 '22:58',
 '22:57',
 '22:57',
 '22:56',
 '22:56',
 '22:55',
 '22:55',
 '22:55',
 '22:55',
 '22:54',
 '22:54',
 '22:54',
 '22:53',
 '22:53',
